In [3]:
import pandas as pd
import numpy as np
import pyreadr
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse as sp
from multiprocessing import Pool

In [1]:
from ps1_funcs import ex1

In [5]:
dta = pyreadr.read_r('AddHealth.RData')

friends_raw = dta['friend']
schools_raw = dta['inschool']

In [6]:
ex1 = ex1(friends_raw, schools_raw)

id_list = ['999999', 77777777, 88888888, 99999999]

data = ex1.clean(id_list=id_list)

ex1.generate_graphs()

In [7]:
selected_schools = ex1.compute_eigenvalues_and_filter_schools(beta=0.1)

In [8]:
selected_schools

[{'school_code': '001',
  'graph': <networkx.classes.graph.Graph at 0x22abab7b010>,
  'largest_eigenvalue': 5.3907622114377425},
 {'school_code': '002',
  'graph': <networkx.classes.graph.Graph at 0x22ababa2850>,
  'largest_eigenvalue': 6.969722026250456},
 {'school_code': '003',
  'graph': <networkx.classes.graph.Graph at 0x22abab7a850>,
  'largest_eigenvalue': 8.371098989675122},
 {'school_code': '004',
  'graph': <networkx.classes.graph.Graph at 0x22abfffa850>,
  'largest_eigenvalue': 6.747114801062647},
 {'school_code': '006',
  'graph': <networkx.classes.graph.Graph at 0x22ac0002850>,
  'largest_eigenvalue': 7.951793014347339},
 {'school_code': '007',
  'graph': <networkx.classes.graph.Graph at 0x22ac0076350>,
  'largest_eigenvalue': 8.818370036392494},
 {'school_code': '009',
  'graph': <networkx.classes.graph.Graph at 0x22abec3e590>,
  'largest_eigenvalue': 8.699934197387098},
 {'school_code': '010',
  'graph': <networkx.classes.graph.Graph at 0x22abed2df90>,
  'largest_eigenval

In [9]:
bonacich_centrality = ex1.compute_bonacich_centrality(beta=0.01)

In [10]:
bonacich_centrality

{'001': array([1.06235752, 1.05406199, 1.0646033 , 1.01062358, 1.01062358,
        1.0422916 , 1.05354767, 1.04210984, 1.06397347, 1.05230401,
        1.08428535, 1.0104211 , 1.03139679, 1.03228329, 1.06428622,
        1.04310914, 1.07281244, 1.04344035, 1.04364054, 1.03269745,
        1.02177033, 1.03208567, 1.07319831, 1.05417982, 1.02104246,
        1.02134251, 1.03114362, 1.04143866, 1.02061976, 1.02062277,
        1.02083842, 1.06255666, 1.02156866, 1.01062557, 1.06480877,
        1.06105223, 1.01061052, 1.01061052, 1.01061052, 1.01061052,
        1.03083232, 1.01030832, 1.03200563, 1.03104773, 1.09260842,
        1.01092608, 1.02165275, 1.03259958, 1.03300716, 1.01092608,
        1.10378309, 1.01103783, 1.01103783, 1.01103783, 1.04354154,
        1.04374979, 1.06321951, 1.03137205, 1.02135696, 1.03156286,
        1.06302534, 1.10422034, 1.0110422 , 1.0110422 , 1.09430915,
        1.02177434, 1.05224728, 1.01052247, 1.01084285, 1.07266678,
        1.01072667, 1.01072667, 1.0831252

In [74]:
# for node_id, attributes in school_graphs['004'].nodes(data=True):
#     print("Node ID:", node_id)
#     print("Attributes:", attributes)